<a href="https://colab.research.google.com/github/hhicks13/portfolio-optimization/blob/main/Lagrange_Vault.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


\begin{array}{ll} \mbox{max} & y^T\mathbf{u} - \frac{1}{2} \gamma y^T M y\\
\mbox{subject to} & {\bf \log y}^Tp = -F,
\end{array}
$\textbf{Solution:}$
> Define the Lagrangian:
$$\mathcal{L}(y,\lambda) = y^T\mathbf{u} - \frac{1}{2}\gamma y^T M y - \lambda(y^Tp + F) $$
> Derive 1st order conditions:
$$\begin{align} \frac{\partial\mathcal{L}}{\partial y} &=& \mathbf{u} -\gamma M y - \gamma p  \overset{\Delta}{=} 0 \\
 \frac{\partial\mathcal{L}}{\partial \lambda} &=& F + y^T {\bf p} \overset{\Delta}{=} 0
 \end{align}$$


$\textbf{Vault Optimization via Lagrange Minimization }$ \
$\textbf{ 1 }$ Let $I_n$ be the identity matrix, and $D$ be the initial deposit.We define the following terms:
$$\begin{align}
\mathbf{u} &\overset{\Delta}{=}& n^{-1}\mathbf{1}_n \\
D &\overset{\Delta}{=}& \texttt{ initial deposit} \\
C_i &\overset{\Delta}{=}& \texttt{ pool i C value} \\
p_i &\overset{\Delta}{=}& \texttt{ pool i free liquidity} \\
w_i &\overset{\Delta}{=}& \texttt{pool i weights for deposit} \\
x_i &\overset{\Delta}{=}& e^{-\frac{w_i D}{p_i}}  \\
y_i &\overset{\Delta}{=}& \log (C_ix_i) \\
&=& \log C_i + \log x_i
\end{align}$$

$\textbf{ 2 }$Matrix Formulation of the log-normal mean estimator $y^T\mathbf{u}$: \
$$
\begin{align}
y^T\mathbf{u} &=& \overline{\log Cx} \\
&=& \frac{\sum_i \log(Cx)_i}{n}\\
\end{align}
$$ \
$\textbf{ 3 }$Matrix Formulation of the log-normal sample covariance estimator $y^TMy$: \
$$\begin{align}
M &=& {I}_n - n^{-1}\mathbf{1}_n\mathbf{1}_n^T \\
y^TMy &=& (n - 1)\sum_i^n({\log(Cx)}_i - \overline{\log (Cx)})^2 \\
\end{align}
$$ \
$\textbf{ 4 }$Due to the fact that all deposits cause a decrease in C value, and all weights must be non-negative and sum to 1, we have the following conditions that must hold:
$$\begin{align}
 -\ln x_i \frac{p_i}{D} &=& w_i \\
 \sum p_i \ln x_i + D &=& 0
 \end{align}$$ \
$\textbf{ 5 }$Proof that condition from $\mathbf{4}$: $\sum_i p_i \log x_i + D = 0$ is equivalent to condition $\sum_i p_i y_i + F = 0 $ \
$$\begin{align}
y &=& \log Cx \\
&=& \log C + \log x \\
y^Tp &=& \sum_i p_i(\log C_i + \log x_i) \\
&=& \log C^T p + \log x^T p \\
F &\overset{\Delta}{=}& D - \log C^T p\\
y^Tp + F &=& \log x^T p + D \\
\end{align}$$





# Auxiliary Functions #

In [5]:
import numpy as np

def u(n):
  return (1/n)*np.ones(n)

def M(n):
  return np.identity(n) + (-1/n)*np.ones(shape=(n,n))

def F(log_c,p,D):
  return D - _log_c.T@p 

def expression(y,p,log_c,D):
  _F = F(log_c,p,D)
  return y.T@p + _F

def weights(y,p,log_c,D):
  wdp =  -1*(y - log_c)
  w = cp.multiply(wdp,p)/D 
  return w

def extract_w(y,p,log_c,D):
  wdp = -1*(y-log_c)
  w = np.multiply(wdp,p)/D
  return w

# Test Case #

In [6]:
import numpy as np
import cvxpy as cp

_c = np.array([1,1000,1])
_log_c = np.log(_c)
_p = np.array([10,10,100]) 
n = 3
_M = M(n)
_u = u(n)
D = 60
y = cp.Variable(n)
gamma = cp.Parameter(nonneg=True)
mean = _u.T@y 
var = cp.quad_form(y, _M)
prob = cp.Problem(cp.Maximize(mean - gamma*var), 
                  [expression(y,_p,_log_c,D) == 0,
                   cp.sum(weights(y,_p,_log_c,D)) == 1,
                   weights(y,_p,_log_c,D) >= 0],)

SAMPLES = 100
risk_data = np.zeros(SAMPLES)
ret_data = np.zeros(SAMPLES)
gamma_vals = np.logspace(-2, 3, num=SAMPLES)
for i in range(SAMPLES):
    gamma.value = gamma_vals[i]
    prob.solve()
    risk_data[i] = cp.sqrt(var).value
    ret_data[i] = mean.value

extract_w(y.value,_p,_log_c,D)

array([-9.86513061e-19,  1.00000000e+00, -7.08923547e-17])